### BOW
: 빈도를 세서 자주 나타나는 낱말로 문서 카테고리를 분리 한다.
ex) 훈련, 사격, 작전 => 군용 문서 / 수술, 환자, 감염, 상처 => 의료 문서

STEP
* (1) 디스크립터 계산
* (2) K-MEANS 알고리즘으로 군집화 
    - 이미지별 특징끼리 분류됨 
* (3) 특징들을 히스토그램화 시킴 => 특정범위에 있으면 해당 물체로 분류

### Train
    - 먼저 ['airplanes', 'Motorbikes' ]에서 학습 후 아래의 Test 행에서 확인 후 
    - 클러스트 개수를 제외한 변수 변경해서 실행해볼것

In [4]:
import cv2
import numpy as np
import os, glob, time

# 각종 변수 선언---①
startT = time.time()                        # 소요시간 측정을 위한 시간 저장
categories =  ['airplanes', 'Motorbikes' ]  # 카테고리 이름 
dictionary_size = 50                        # 사전 크기, 클러스터 갯수 
base_path = "../img/101_ObjectCategories/"  # 학습 이미지 기본 경로 
dict_file = 'plane_bike_dict.npy'         # 사전 객체 저장할 파일 이름 
svm_model_file = 'plane_bike_svm.xml'     # SVM 모델 객체 저장할 파일 이름 

# 추출기와 BOW 객체 생성 --- ②
detector = cv2.xfeatures2d.SIFT_create()    # 추출기로 SIFT 생성 
matcher = cv2.BFMatcher(cv2.NORM_L2)        # 매칭기로 BF 생성
bowTrainer = cv2.BOWKMeansTrainer(dictionary_size) # KMeans로 구현된 BWOTrainer 생성
bowExtractor = cv2.BOWImgDescriptorExtractor(detector, matcher) # 히스토그램 계산할 BOW추출기 생성

# 특징 디스크립터를 KMeansTrainer에 추가---③
train_paths = []                            # 훈련에 사용할 모든 이미지 경로 
train_labels = []                           # 학습 데이타 레이블
print('Adding descriptor to BOWTrainer...')
for idx, category in enumerate(categories): # 카테고리 순회
    dir_path = base_path + category          
    img_paths = glob.glob(dir_path +'/*.jpg') 
    img_len = len(img_paths)
    for i, img_path in enumerate(img_paths): # 카테고리 내의 모든 이미지 파일 순회
        train_paths.append(img_path)        
        train_labels.append(idx)            # 학습 데이타 레이블, 0 또는 1 
        img = cv2.imread(img_path)          
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # 특징점과 특징 디스크립터 추출 및 bowTrainer에 추가 ---④
        kpt, desc= detector.detectAndCompute(gray, None) 
        bowTrainer.add(desc)                
        print('\t%s %d/%d(%.2f%%)' \
              %(category,i+1, img_len, (i+1)/img_len*100), end='\r')
    print()
print('Adding descriptor completed...')

# KMeans 클러스터로 군집화하여 시각 사전 생성 및 저장---⑤
print('Starting Dictionary clustering(%d)... \
        It will take several time...'%dictionary_size)
dictionary = bowTrainer.cluster() # 군집화로 시각 사전 생성  
np.save(dict_file, dictionary)    # 시각 사전 데이타(넘파일)를 파일로 저장
print('Dictionary Clustering completed...dictionary shape:',dictionary.shape)

# 시각 사전과 모든 이미지의 매칭점으로 히스토그램 계산---⑥
bowExtractor.setVocabulary(dictionary)      # bowExtractor에 시각 사전 셋팅 
train_desc = []                             # 학습 데이타 
for i, path in enumerate(train_paths):      # 모든 학습 대상 이미지 순회
    img = cv2.imread(path)                  # 이미지 읽기 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    # 매칭점에 대한 히스토그램 계산 --- ⑦
    hist = bowExtractor.compute(gray, detector.detect(gray)) 
    train_desc.extend(hist)                 
    print('Compute histogram training set...(%.2f%%)'\
                    %((i+1)/len(train_paths)*100),end='\r')
print("\nsvm items", len(train_desc), len(train_desc[0]))

# 히스토그램을 학습데이타로 SVM 훈련 및 모델 저장---⑧
print('svm training...')
svm = cv2.ml.SVM_create()
svm.trainAuto(np.array(train_desc), cv2.ml.ROW_SAMPLE, np.array(train_labels))
svm.save(svm_model_file)
print('svm training completed.')
print('Training Elapsed: %s'\
        %time.strftime('%H:%M:%S', time.gmtime(time.time()-startT)))

# 원래의 이미지로 테스트 --- ⑨
print("Accuracy(Self)")
for label, dir_name in enumerate(categories):
    labels = []
    results = []
    img_paths = glob.glob(base_path + '/'+dir_name +'/*.*')
    for img_path in img_paths:
        labels.append(label)
        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        feature = bowExtractor.compute(gray, detector.detect(gray))
        ret, result = svm.predict(feature)
        resp = result[0][0]
        results.append(resp)

    labels = np.array(labels)
    results = np.array(results)
    err = (labels != results)
    err_mean = err.mean()
    print('\t%s: %.2f %%' % (dir_name, (1 - err_mean)*100))

Adding descriptor to BOWTrainer...
	airplanes 800/800(100.00%)
	Motorbikes 798/798(100.00%)
Adding descriptor completed...
Starting Dictionary clustering(50)...         It will take several time...
Dictionary Clustering completed...dictionary shape: (50, 128)
Compute histogram training set...(100.00%)
svm items 1598 50
svm training...
svm training completed.
Training Elapsed: 00:02:10
Accuracy(Self)
	airplanes: 96.38 %
	Motorbikes: 95.36 %


#### test

In [ ]:
import cv2
import numpy as np

categories =  ['airplanes', 'Motorbikes' ]
dict_file = 'plane_bike_dict.npy'
#dict_file = './plane_bike_dict_4000.npy'
svm_model_file = 'plane_bike_svm.xml'
#svm_model_file = './plane_bike_svm_4000.xml'

# 테스트 할 이미지 경로 --- ①
imgs = ['../img/aircraft.jpg','../img/jetstar.jpg', 
        '../img/motorcycle.jpg', '../img/motorbike.jpg']

# 특징 추출기(SIFT) 생성 ---②
detector = cv2.xfeatures2d.SIFT_create()
# BOW 추출기 생성 및 사전 로딩 ---③
bowextractor = cv2.BOWImgDescriptorExtractor(detector, \
                                cv2.BFMatcher(cv2.NORM_L2))
bowextractor.setVocabulary(np.load(dict_file))
# 훈련된 모델 읽어서 SVM 객체 생성 --- ④
svm  = cv2.ml.SVM_load(svm_model_file)

# 4개의 이미지 테스트 
for i, path in enumerate(imgs):
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 테스트 이미지에서 BOW 히스토그램 추출 ---⑤
    hist = bowextractor.compute(gray, detector.detect(gray))
    # SVM 예측 ---⑥
    ret, result = svm.predict(hist)
    # 결과 표시 
    name = categories[int(result[0][0])]
    txt, base = cv2.getTextSize(name, cv2.FONT_HERSHEY_PLAIN, 2, 3)
    x,y = 10, 50
    cv2.rectangle(img, (x,y-base-txt[1]), (x+txt[0], y+txt[1]), (30,30,30), -1)
    cv2.putText(img, name, (x,y), cv2.FONT_HERSHEY_PLAIN, \
                                 2, (0,255,0), 2, cv2.LINE_AA)
    cv2.imshow(path, img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Histogram of Oriented Gradient(HOG)

(1) 보행자 검출을 목적으로 만들어 졌다. 
(2) SIFT, SURF, ORB 등에 비해 전체적인 모양을 특징으로 표현하기 용이하다. 

순서
1. 인식하고자 하는 영역을 자른다. (window)
2. 소벨 필터를 이용해서 엣지의 기울기 gx, gy를 구한다.  
3. 8 x 8셀 나누어 기울기의 방향과 크기를 계산한다. 
4. 2배 크기의 블록으로 셀을 노멀라이즈 

=> 픽셀 차이를 이용해서 기울기를 구한다

In [2]:

"""
img = cv2.imread('img.png')
img =  np.float(img)

gx = cv.Sobel(img, cv.CV_32F, 1, 0)
gy = cv.Sobel(img, cv.CV_32F, 0, 1)
magnitude, angle = cv.cartToPolar(gx, gy)
    - magnitude : 크기 
    - angle : 기울기의 방향

"""

IndentationError: unexpected indent (<ipython-input-2-0b9d6cef23ae>, line 7)

### HOG - SVM 보행자 검출
* cv2.HOGDescriptor_getDefaultPeopleDetector() : 64 x 128 윈도 크기로 훈련된 모델 

In [7]:
import cv2

# default 디덱터를 위한 HOG 객체 생성 및 설정--- ①
hogdef = cv2.HOGDescriptor()
hogdef.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# dailer 디덱터를 위한 HOG 객체 생성 및 설정--- ②
hogdaim  = cv2.HOGDescriptor((48,96), (16,16), (8,8), (8,8), 9)
hogdaim.setSVMDetector(cv2.HOGDescriptor_getDaimlerPeopleDetector())

cap = cv2.VideoCapture('../img/walking.avi')
mode = True  # 모드 변환을 위한 플래그 변수 
print('Toggle Space-bar to change mode.')
while cap.isOpened():
    ret, img = cap.read()
    if ret :
        if mode:
            # default 디텍터로 보행자 검출 --- ③
            found, _ = hogdef.detectMultiScale(img)
            for (x,y,w,h) in found:
                cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,255))
        else:
            # daimler 디텍터로 보행자 검출 --- ④
            found, _ = hogdaim.detectMultiScale(img)
            for (x,y,w,h) in found:
                cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0))
        cv2.putText(img, 'Detector:%s'%('Default' if mode else 'Daimler'), \
                        (10,50 ), cv2.FONT_HERSHEY_DUPLEX,1, (0,255,0),1)
        cv2.imshow('frame', img)
        key = cv2.waitKey(1) 
        if key == 27:
            break
        elif key == ord(' '): 
            mode = not mode
    else:
        break
cap.release()
cv2.destroyAllWindows()

Toggle Space-bar to change mode.


### 한니발 마스크

In [1]:
import cv2
import numpy as np

# 마스크 이미지 읽기 
face_mask = cv2.imread('img/mask_hannibal.png')
h_mask, w_mask = face_mask.shape[:2]
# 얼굴 검출기 생성
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	# 얼굴 영역 검출
    face_rects = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in face_rects:
        if h > 0 and w > 0:
        		# 마스크 위치 보정
            x = int(x + 0.1*w)
            y = int(y + 0.4*h)
            w = int(0.8 * w)
            h = int(0.8 * h)

            frame_roi = frame[y:y+h, x:x+w]
            # 마스크 이미지를 얼굴 크기에 맞게 조정 
            face_mask_small = cv2.resize(face_mask, (w, h), \
                                interpolation=cv2.INTER_AREA)
			# 마스크 이미지 합성
            gray_mask = cv2.cvtColor(face_mask_small, cv2.COLOR_BGR2GRAY)
            ret, mask = cv2.threshold(gray_mask, 50, 255, cv2.THRESH_BINARY)
            mask_inv = cv2.bitwise_not(mask)
            masked_face = cv2.bitwise_and(face_mask_small, face_mask_small,\
                                         mask=mask)
            masked_frame = cv2.bitwise_and(frame_roi, frame_roi, mask=mask_inv)
            frame[y:y+h, x:x+w] = cv2.add(masked_face, masked_frame)

    cv2.imshow('Hanibal Mask', frame)
    if cv2.waitKey(1) == 27:
        break
cap.release()
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'

In [4]:
import cv2
import numpy as np

# 얼굴과 눈동자 검출기 생성
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

# 렌즈 왜곡 효과 함수
def distortedMap(rows, cols, type=0):
    map_y, map_x = np.indices((rows, cols), dtype=np.float32)
    # 렌즈 효과
    ## 렌즈 효과, 중심점 이동
    map_lenz_x = (2*map_x - cols)/cols
    map_lenz_y = (2*map_y - rows)/rows
    ## 렌즈 효과, 극좌표 변환
    r, theta = cv2.cartToPolar(map_lenz_x, map_lenz_y)
    if type==0:
    ## 볼록 렌즈 효과 매핑 좌표 연산
        r[r< 1] = r[r<1] **3  
    else:
    ## 오목 렌즈 효과 매핑 좌표 연산
        r[r< 1] = r[r<1] **0.5
    ## 렌즈 효과, 직교 좌표 복원
    mapx, mapy = cv2.polarToCart(r, theta)
    ## 렌즈 효과, 좌상단 좌표 복원
    mapx = ((mapx + 1)*cols)/2
    mapy = ((mapy + 1)*rows)/2
    return (mapx, mapy)

# 얼굴 검출 함수
def findFaces(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    face_coords = []
    for (x,y,w,h) in faces:
        face_coords.append((x, y, w, h))
    return face_coords
# 눈 검출 함수
def findEyes(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    eyes_coords = []
    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray )
        for(ex,ey,ew,eh) in eyes:
            eyes_coords.append((ex+x,ey+y,ew,eh))
    return eyes_coords


cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 480)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 320)

while True:
    ret, frame = cap.read()
    img1 = frame.copy()
    img2 = frame.copy()
    # 얼굴 검출해서 오목/볼록 렌즈 효과로 왜곡 적용
    faces = findFaces(frame)
    for face in faces:
        x,y,w,h = face
        mapx, mapy = distortedMap(w,h, 1)
        roi = img1[y:y+h, x:x+w]
        convex = cv2.remap(roi,mapx,mapy,cv2.INTER_LINEAR)
        img1[y:y+h, x:x+w] = convex
    # 눈 영역 검출해서 볼록 렌즈 효과로 왜곡 적용
    eyes = findEyes(frame)
    for eye in eyes :
        x,y,w,h = eye
        mapx, mapy = distortedMap(w,h)
        roi = img2[y:y+h, x:x+w]
        convex = cv2.remap(roi,mapx,mapy,cv2.INTER_LINEAR)
        img2[y:y+h, x:x+w] = convex
    # 하나의 이미지로 병합해서 출력
    merged = np.hstack((frame, img1, img2))
    cv2.imshow('Face Distortion', merged)
    if cv2.waitKey(1) == 27:
        break
cv2.destroyAllWindows()
        

AttributeError: 'NoneType' object has no attribute 'copy'